In [1]:
from datetime import datetime, timedelta
from pytz import timezone
import pytz
from dateutil.relativedelta import *

## Time zones


#### ref.



[document for pytz](http://pytz.sourceforge.net/#localized-times-and-date-arithmetic)

#### Now

In [2]:
datetime.now()

datetime.datetime(2018, 10, 2, 15, 41, 16, 803133)

In [3]:
datetime.utcnow()

datetime.datetime(2018, 10, 2, 22, 41, 16, 830692)

### Define timezone

In [4]:
pst = timezone('US/Pacific-New')
cst = timezone('Asia/Shanghai') # china standard time
utc = pytz.utc

#### Get a list of time zones

In [5]:
# pytz.all_timezones

### "Localize": adding time zone information to datetime
adding info that "this is in utc time"

In [6]:
## adding info that "this is in utc time"
utc_time = utc.localize(datetime(2018, 8, 23, 22, 52, 8, 797172))

In [7]:
utc_time

datetime.datetime(2018, 8, 23, 22, 52, 8, 797172, tzinfo=<UTC>)

In [8]:
utc_time.tzinfo # this is the timezone information that can be used to "loc`balize" a naive datetime object

<UTC>

This is the timezone information that can be used to "localize" a naive datetime object

### convert timezones

In [9]:
pst_time = utc_time.astimezone(pst)
pst_time

datetime.datetime(2018, 8, 23, 15, 52, 8, 797172, tzinfo=<DstTzInfo 'US/Pacific-New' PDT-1 day, 17:00:00 DST>)

In [10]:
print(pst_time)

2018-08-23 15:52:08.797172-07:00


### Time zone-safe comparison

In [11]:
print(utc_time)
print(pst_time)

utc_time == pst_time

2018-08-23 22:52:08.797172+00:00
2018-08-23 15:52:08.797172-07:00


True

## Truncate


### Truncate datetime

In [12]:
pst_time.replace(hour=0, minute=0, second=0, microsecond=0)

datetime.datetime(2018, 8, 23, 0, 0, tzinfo=<DstTzInfo 'US/Pacific-New' PDT-1 day, 17:00:00 DST>)

In [13]:
utc_time.replace(hour=0, minute=0, second=0, microsecond=0)

datetime.datetime(2018, 8, 23, 0, 0, tzinfo=<UTC>)

In [14]:
print('time1 :  ' + str(pst_time))
pst_time2 = utc.localize(datetime(2018, 8, 23, 20, 52, 8, 0)).astimezone(pst)
print('time2 :  ' + str(pst_time))

time1 :  2018-08-23 15:52:08.797172-07:00
time2 :  2018-08-23 15:52:08.797172-07:00


In [15]:
## After truncation should be the same
pst_time.replace(hour=0, minute=0, second=0, microsecond=0) \
    == pst_time2.replace(hour=0, minute=0, second=0, microsecond=0)

True

### Truncate timedelta

In [16]:
def truncate_microseconds(delta):
    '''
    https://stackoverflow.com/questions/18470627/python-timedelta-remove-microseconds
    '''
    return delta - timedelta(microseconds=delta.microseconds)

In [17]:
print(pst_time - pst_time2)

2:00:00.797172


In [18]:
print(truncate_microseconds(pst_time - pst_time2))

2:00:00


## Sum timedelta

In [19]:
print(sum([timedelta(1) , timedelta(2, seconds = 2)], timedelta()))

3 days, 0:00:02


## Calender timedelta (relative time delta)


See answer on [stackoverflow](https://stackoverflow.com/questions/35066588/is-there-a-simple-way-to-increment-a-datetime-object-one-month-in-python)

In [20]:
from dateutil.relativedelta import *

#### Month to date

In [21]:
pst = timezone('US/Pacific-New')
today = pst.localize(datetime.now())
print(today)

2018-10-02 15:41:17.237209-07:00


In [22]:
month_to_date = today + relativedelta(months=-1)
print(month_to_date)

2018-09-02 15:41:17.237209-07:00


#### "The second Tuesday next month"

In [23]:
next_meeting = today + relativedelta(months=+1, day=1, weekday=TU(2))
print(next_meeting)

2018-11-13 15:41:17.237209-07:00


## Timestamp to string

#### Cheat sheet for most seen formats

In [24]:
ts = datetime.now()

In [25]:
ts.strftime("%Y-%b(%B)-%d %H:%M:%S")

'2018-Oct(October)-02 15:41:17'

In [26]:
ts.strftime("%Y-%m-%d (%a / %A) %H:%M:%S")

'2018-10-02 (Tue / Tuesday) 15:41:17'

In [27]:
utc.localize(ts).strftime("%Y-%m-%d %I:%M (%p)")

'2018-10-02 03:41 (PM)'

In [28]:
utc.localize(ts).strftime("%Y-%m-%d %I:%M %p (%Z)")

'2018-10-02 03:41 PM (UTC)'

## String to timestamp

In [78]:
ts_str = utc.localize(ts).astimezone(pst).strftime("%Y-%m-%d %I:%M %p (%z)")
ts2 = datetime.strptime(ts_str,"%Y-%m-%d %I:%M %p (%z)")

In [79]:
ts_str 

'2018-10-02 08:41 AM (-0700)'

#### Hour (as a number) to string

1. Direct formating
2. `print` timedelta
3. make a trivial datetime and `strftime`

In [29]:
"%02d:00" % 1

'01:00'

In [30]:
h=15.3
print(timedelta(hours = h))

15:18:00


In [34]:
td = timedelta(hours = h)

In [55]:
def timedelta2str(td):
    '''
    if the str(td) does not work
    '''
    (h, res) = divmod(int(td.total_seconds()), 3600)
    (m, s) = divmod(res, 60)
    return '{:02}:{:02}:{:02}'.format(h,m,s)

In [53]:
timedelta2str(td)

'15:18:00'

In [32]:
(datetime(1,1,1) + timedelta(hours = h)).strftime("%I:%M %p")

'03:18 PM'

## Unix timestamp

(Bigint, 1494032110 stands for 2017-05-06 08:55:10 CST)

In [33]:
utc.localize(datetime.utcfromtimestamp(1494032110)).astimezone(cst)

datetime.datetime(2017, 5, 6, 8, 55, 10, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>)